# Центральность по посредничеству.

## Загрузка графа друзей через VK API

In [89]:
# Текущий каталог где находятся файлы
FILES_DIR = "C:\\Projects\\friends-graph"

# Сначала устанавливаем библиотеки для работы с графами, vk api и загрузкой файлов в yaml
import sys
import os

REQ_PATH = os.path.join(FILES_DIR, "requirements.txt")
!{sys.executable} -m pip install -r $REQ_PATH

In [78]:
# Может быть ошибка: что библиотека PyYAML уже установлена

In [79]:
import vk_api
import networkx as nx  # библиотека графов 

MAX_DEEP = 5  # Максимальная глубина графа
MAX_FRIENDS = 30 # Максимальное число друзей которое может обработать vk_api для данного узла
VK_LOGIN = 'kirainluck@gmail.com'
VK_TOKEN = 'ce373d45c3f69d93022b12d3e9af02916e22becd049aaeeb40de1c19e7a65a3e0e58945b1d547beab9e0b'
VK_IDS = [
    559640345, 29759351, 120435774, 78517271, 55564167,
    202514864, 79818398, 18214699, 25689500, 322913600,
    32418429, 32784038, 55355150, 65817487, 76637395,
    31710423, 557916962, 44907648
]

GRAPH_FILENAME = os.path.join(FILES_DIR, 'friends_graph_full.yaml')

In [80]:
def fill_graph(vk, graph, id, deep=1):
    """
    Функция загрузки графа
    
    """
    max_cnt = MAX_FRIENDS
    # обычно все друзья на первом уровне
    if deep == 1:
        max_cnt = 500
    try:
        response = vk.friends.get(user_id=id, count=max_cnt)
    except vk_api.exceptions.ApiError as e:
        print(f"VK api error on id {id} and deep {deep}\n {e}")
        return

    fr_in_lst = False
    for fr_id in response['items']:
        graph.add_edge(id, fr_id)
        if fr_id in VK_IDS:
            fr_in_lst = True
    if (not fr_in_lst) and (deep < MAX_DEEP):
        deep += 1
        for fr_id in response['items']:
            fill_graph(vk, graph, fr_id, deep)

In [81]:
# Загружаем граф рекурсивно с максимальной глубиной MAX_DEEP

In [82]:
friends_graph = None

if not os.path.exists(GRAPH_FILENAME):
    vk_session = vk_api.VkApi(login=VK_LOGIN, token=VK_TOKEN)
    vk = vk_session.get_api()
    friends_graph = nx.Graph()
    for id in VK_IDS:
        fill_graph(vk, friends_graph, id)
        
    # Записываем результат в файл
    nx.readwrite.nx_yaml.write_yaml(friends_graph, GRAPH_FILENAME)
else:
    # Загружаем граф из файла
    friends_graph = nx.readwrite.nx_yaml.read_yaml(GRAPH_FILENAME)

In [91]:
print(f"Number of nodes: {friends_graph.number_of_nodes()}") # долго считает

Number of nodes: 85758


## Вычисление центральности по посредничеству

In [92]:
between_cent = {key:value for (key, value) in nx.betweenness_centrality(friends_graph, k=10).items() if key in VK_IDS}
between_cent

{18214699: 0.0012710629114378983,
 25689500: 4.834008558424208e-05,
 29759351: 0.0007197072665391756,
 32418429: 0.0,
 32784038: 0.0,
 55355150: 0.0,
 55564167: 0.0018071174348656615,
 78517271: 0.0011774020744635978,
 79818398: 0.0005252959583475647,
 120435774: 0.002440480383129057,
 202514864: 0.002597780659422589,
 322913600: 0.0,
 559640345: 2.8504978645905976e-05}

In [93]:
import operator

between_cent_id = max(between_cent.items(), key=operator.itemgetter(1))

In [94]:
between_cent_id # id и значение центральности

(202514864, 0.002597780659422589)

In [ ]:
# Центральный по посредничеству - Артем Осин